# Tutorial on Automation

## Jobflow
We will use [jobflow](https://github.com/materialsproject/jobflow) to write and execute workflows on our local machines. One can also start these computations on supercomputers.


## Install Software

We start by installing all relevant software. We will use jobflow but also two typical software from computational materials science. [ASE](https://gitlab.com/ase/ase) and [pymatgen](https://pymatgen.org/). pymatgen works especially well together with jobflow as both have been developed by the [Materials Project](https://materialsproject.org/).

In [ ]:
%%capture
!pip install jobflow

In [ ]:
%%capture
!pip install ase

In [ ]:
%%capture
!pip install pymatgen

## Workflow to compute the equation of state (e.g., to compute a bulk modulus)

In this first tutorial, we will build flows and jobs to automatize the computation of the equation of state. This tutorial has been inspired by the equation of state tutorial from ASE: [https://wiki.fysik.dtu.dk/ase/tutorials/eos/eos.html](https://wiki.fysik.dtu.dk/ase/tutorials/eos/eos.html) . Please also check out this nice paper on equations of state: [K. Latimer, S. Dwaraknath, K. Mathew, D. Winston, K. A. Persson, npj Comput Mater 2018, 4, 40.](https://doi.org/10.1038/s41524-018-0091-x) You can use each flavor with the help of pymatgen.


Let's have a look how we would compute the results without workflow. This is only possible if we have a very fast calculator (i.e., some object to get energies and forces from ab initio theories or force fields) and small unit cells.

We will start by constructing a simple unit cell with the Atoms object in ASE: Al in fcc structure type. The Atoms object in ASE is used to represent the structure. It can also be directly connected to electronic energies and forces when a calculator is added.

In [ ]:
from ase.build import bulk
atoms=bulk("Ag","fcc",3.9)

In [ ]:
print(atoms)

You can visualize the structure when you uncomment the next lines.

In [ ]:
#from ase.visualize import view
#view(atoms)

Instead of using an ASE Atoms object, we will later on also use Structure objects from pymatgen. They are very handy as they can be easily transformed to dict representations in databases. There are conversion tools available in pymatgen. Let's create a Structure object from an Atoms object. This is now just structural information.

In [ ]:
from pymatgen.io.ase import AseAtomsAdaptor

structure=AseAtomsAdaptor().get_structure(atoms)
print(structure)
print(structure.as_dict())

### Effective Medium Theory
To calculate energies and forces, we will use effective-medium theory as described in [K. W. Jacobsen, J. K. Norskov, M. J. Puska, Phys. Rev. B 1987, 35, 7423–7442.
](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.35.7423). This is mainly to be able to perform the computations without high-performance computers and still have somewhat reasonable results for simple solids such as Al.  The method is based on an ansatz that assumes that the total electron density can be expressed as a sum of the electron densities of individual atoms. The electron densities of the individual atoms are calculated separately assuming that each atom is embedded in a homogeneous electron gas. It's not necessary to understand this approximation in detail to perform the exercises. We really just need a simple calculator to get energies and forces... Our typical workflows use DFT.


Let's add a calculator to the Atoms object.

In [ ]:
from ase.calculators.emt import EMT
atoms.calc=EMT()

### Optimization of the structure
We will use a standard optimizer from ASE (BFGS) here to perform the optimization. We add an additional Filter to also optimize the cell parameters and not only the atomic positions.

In [ ]:
from ase.optimize import BFGS
from ase.constraints import ExpCellFilter
print(atoms)
opt = BFGS(ExpCellFilter(atoms))
opt.run(fmax=0.00001)
print(atoms)

As you can see the cell parameters have slightly changed. The atomic positions will not really change due to the high symmetry.

In [ ]:
print(atoms.calc.results)

After this optimization of the structure, we will now scale the cell parameters to get our Energy-Volume curve, i.e. we evaluate the energy at different volumes. We will first get the current cell parameters:

In [ ]:
cell=atoms.get_cell()
print(cell)

Let's try the scaling:

In [ ]:
new_cell=cell*1.01
print(new_cell)
print(cell)

Now, let's perform the energy volume curve computation. The optimizations that we will perform will now be at constant volume. We will collect the volumes and energies.

In [ ]:
import numpy as np
from ase.io.trajectory import Trajectory
from copy import deepcopy

cell = atoms.get_cell()

volumes=[]
energies=[]
for x in [0.95, 0.97, 0.99, 1.00, 1.01, 1.03, 1.05]:
    optimized_atoms = deepcopy(atoms)
    optimized_atoms.set_cell(cell * x, scale_atoms=True)
    opt = BFGS(ExpCellFilter(optimized_atoms, constant_volume=True))
    opt.run(fmax=0.00001)
    volumes.append(optimized_atoms.get_volume())
    energies.append(optimized_atoms.get_potential_energy())

print(volumes)
print(energies)

Let's do a plot of the equation of states now.

In [ ]:
from ase.units import kJ
from ase.eos import EquationOfState

eos = EquationOfState(volumes, energies)
v0, e0, B = eos.fit()
print(B, 'eV Angstrom**(-3)')
print(B / kJ * 1.0e24, 'GPa')
eos.plot('Ag-eos.png')

This is fine. You can check out the DFT equations of state here: [https://materialsproject.org/materials/mp-124?chemsys=Ag#equations_of_state](https://materialsproject.org/materials/mp-124?chemsys=Ag#equations_of_state) The volume is different by 1 Angstrom³ and Bulk modulus is off by a factor of 10 depending on the equation of state.

We now have a workflow that looks roughly like this:

![Workflow](./workflow.png)

Let's work on an atomic job to optimize the structure. We will use a Structure object to carry the structural information as it can be easily transformed into a dict. We need this as we will save our results in databases that build upon the json and yaml format. This is more complicated for an atoms object. Check out [https://materialsproject.github.io/maggma/concepts](maggma) to learn more.

To write an atomic job in jobflow, we have to inherit from the Maker class and adapt the make method. Please don't forget the decorators as they are central for the execution of the workflow. You can find more about jobs here: [https://materialsproject.github.io/jobflow/tutorials/3-defining-jobs.html](https://materialsproject.github.io/jobflow/tutorials/3-defining-jobs.html)
Such a Maker class will also get the decorator @dataclass . This will add special methods to the class. See here for more info: [https://docs.python.org/3/library/dataclasses.html](https://docs.python.org/3/library/dataclasses.html)

In [ ]:
from jobflow.core.maker import Maker

In [ ]:
from dataclasses import dataclass
from jobflow import job, Maker
from pymatgen.core.structure import Structure

@dataclass
class OptimizeJob(Maker):
    """
    Class to carry out a full optimization of a structure with the EMT calculator

    Parameters
    ----------
    name
        Name of the job.
    """
    name: str = "EMT-Optimization"

    @job
    def make(self, structure: Structure):
        pass

We add a "name" to the maker which will later on help us identify which job is running. And the make method takes a structure as an input. In the next step, we simply add steps to perform the optimization. Thus, we take the same classes, methods and functions as before:

In [ ]:
from dataclasses import dataclass
from jobflow import job
from pymatgen.core.structure import Structure


@dataclass
class OptimizeJob(Maker):
    """
    Class to carry out a full optimization of a structure with the EMT calculator

    Parameters
    ----------
    name
        Name of the job.
    """
    name: str = "EMT-Optimization"

    @job
    def make(self, structure: Structure):
        adaptor = AseAtomsAdaptor()
        atoms = adaptor.get_atoms(structure)  # get an atoms object
        atoms.calc = EMT()  # add emt as a calculator

        opt = BFGS(ExpCellFilter(atoms))  # optimize the structure including cell parameters
        opt.run(fmax=0.00001)  # run the optimization


We have now optimized our structure. We might want to add fmax as a parameter of our Maker to change the default values. We will not do this for all potential settings as it just takes a lot of time to implement and we just want to find out how it works and not do a lot of work...

In [ ]:
@dataclass
class OptimizeJob(Maker):
    """
    Class to carry out a full optimization of a structure with the EMT calculator

    Parameters
    ----------
    name
        Name of the job.
    fmax
        float that determines the forces after optimization
    """
    name: str = "EMT-Optimization"
    fmax: float = 0.00001

    @job
    def make(self, structure: Structure):
        adaptor = AseAtomsAdaptor()
        atoms = adaptor.get_atoms(structure)
        atoms.calc = EMT()
        opt = BFGS(ExpCellFilter(atoms))
        opt.run(fmax=self.fmax)


So far however, our results will be not transferred between jobs. We need to take care of this next. For now, it will be enough to save the optimized structure and the energy at the end. You can also save forces, stresses and so on... People typically develop very large schema. This is again just a lot of work and will not help us to understand the automation itself. It's again just important that all outputs are either a dict or have the as_dict() method.

In [ ]:
@dataclass
class OptimizeJob(Maker):
    """
    Class to carry out a full optimization of a structure with the EMT calculator

    Parameters
    ----------
    name
        Name of the job.
    fmax
        float that determines the forces after optimization
    """
    name: str ="EMT-Optimization"
    fmax: float=0.00001

    @job
    def make(self, structure: Structure):
        adaptor=AseAtomsAdaptor()
        atoms=adaptor.get_atoms(structure)
        atoms.calc=EMT()
        opt = BFGS(ExpCellFilter(atoms))
        opt.run(fmax=self.fmax)

        out_structure=adaptor.get_structure(atoms)

        return out_structure



We will add another parameter to be able to do a constant volume optimization with the same maker.

In [ ]:
@dataclass
class OptimizeJob(Maker):
    """
    Class to carry out a full optimization of an atoms object

    Parameters
    ----------
    name
        Name of the job.
    fmax
        float that determines the forces after optimization
    constant_volume
        bool that if true will enforce an optimization with constant volume
    """
    name: str ="EMT-Optimization"
    fmax: float=0.00001
    constant_volume: bool= False

    @job
    def make(self, structure: Structure):
        adaptor=AseAtomsAdaptor()
        atoms=adaptor.get_atoms(structure)
        atoms.calc=EMT()
        opt = BFGS(ExpCellFilter(atoms, constant_volume=self.constant_volume))
        opt.run(fmax=self.fmax)

        out_structure=adaptor.get_structure(atoms)

        results={}
        results["optimized_structure"]=out_structure
        results["input_structure"]=structure
        results["energy"]=atoms.get_potential_energy()
        return results

For test purposes, we will simply now chain two optimizations, put them in a Flow (i.e., the object for workflows) and see what happens. We also always have to define an output for the flow. Very often, this is the output of the last job in the flow.

In [ ]:
from jobflow.managers.local import run_locally
from jobflow import Flow

atoms=bulk("Ag","fcc",4)
structure=AseAtomsAdaptor().get_structure(atoms)
maker_fullopt=OptimizeJob()
maker_opt_constant_vol=OptimizeJob(constant_volume=True)
job1=maker_fullopt.make(structure=structure)
job2=maker_opt_constant_vol.make(structure=job1.output["optimized_structure"])

flow=Flow([job1,job2],output=job2.output)

response=run_locally(flow)
print(response)


This is nice but of course a bit useless. Btw, you can also create folders for each atomic job in your workflow by using one of the parameters in run_locally.

We now need to build a job that creates structures with different volumes, runs the optimizations with constant volume, and one job that can assess the results!

We will start by using a job that is called get_ev_curve. It will replace itself with a Response object that will start a completely new workflow. As outputs, we will collect all optimized structures and the corresponding energies!

In [ ]:
from jobflow import Response, job


@job
def get_ev_curve(structure: Structure, volumes=[0.95, 0.97, 0.99, 1.00, 1.01, 1.03, 1.05]):
    groundstate_vol = structure.volume
    const_vol_maker = OptimizeJob(constant_volume=True)
    job_list = []
    outputs = {
        "optimized_structures": [],
        "energies": [],
    }
    for vol_inc in volumes:
        struct_here = deepcopy(structure)
        struct_here.scale_lattice(groundstate_vol * vol_inc)
        relax_job = const_vol_maker.make(struct_here)
        job_list.append(relax_job)
        outputs["optimized_structures"].append(relax_job.output["optimized_structure"])
        outputs["energies"].append(relax_job.output["energy"])
    return Response(replace=Flow(job_list, output=outputs))



We can now build a flow including a full optmization and this code to start the constant volume optimizations

In [ ]:
atoms = bulk("Ag", "fcc", 4)
structure = AseAtomsAdaptor().get_structure(atoms)
maker_fullopt = OptimizeJob()
maker_opt_constant_vol = OptimizeJob(constant_volume=True)
job1 = maker_fullopt.make(structure=structure)
job2 = get_ev_curve(structure=job1.output["optimized_structure"])

flow = Flow([job1, job2], output=job2.output)

response = run_locally(flow)

We will now write a tiny job that will collect all results. It will start from a list of structures and energies and then plot an EV curve to a file and return the most important results.

In [ ]:
@job
def get_results_ev_curve(list_structures, list_energies):
    list_volumes = []
    for struct in list_structures:
        list_volumes.append(struct.volume)

    eos = EquationOfState(list_volumes, list_energies)
    v0, e0, B = eos.fit()
    eos.plot('eos.png')

    results = {}
    results["V0"] = v0
    results["e0"] = e0
    results["B"] = B

    return Response(results)

Now, we will combine all jobs to a larger workflow and check out the output

In [ ]:
atoms = bulk("Ag", "fcc", 4)
structure = AseAtomsAdaptor().get_structure(atoms)
maker_fullopt = OptimizeJob()
job1 = maker_fullopt.make(structure=structure)
job2 = get_ev_curve(structure=job1.output["optimized_structure"])
job3 = get_results_ev_curve(list_structures=job2.output["optimized_structures"], list_energies=job2.output["energies"])
flow = Flow([job1, job2, job3], output=job3.output)

response = run_locally(flow)
print(response[flow.jobs[-1].uuid][1].output)

It works. We get the same results!


I will just show you now how to validate your data with a [pydantic](https://docs.pydantic.dev) schema. This allows to describe your outputs and make sure that you also get all relevant data in the right format. Again, this is only one tiny example. In practice, you would add many more options! This is how such a schema can look in practice: [https://github.com/materialsproject/atomate2/blob/main/src/atomate2/lobster/schemas.py](https://github.com/materialsproject/atomate2/blob/main/src/atomate2/lobster/schemas.py)

In [ ]:
from pydantic import BaseModel, Field


class EOS_Schema(BaseModel):
    """Collection to store computational settings for the phonon computation."""

    V0: float = Field("Volume in Angstrom**3")
    e0: float = Field("Energy in eV")
    B: float = Field(
        "Bulk modulus",
    )


@job(output_schema=EOS_Schema)
def get_results_ev_curve(list_structures, list_energies):
    list_volumes = []
    for struct in list_structures:
        list_volumes.append(struct.volume)

    eos = EquationOfState(list_volumes, list_energies)
    v0, e0, B = eos.fit()
    eos.plot('eos.png')

    results = {}
    results["V0"] = v0
    results["e0"] = e0
    results["B"] = B
    schema = EOS_Schema(**results)
    return Response(schema)

let's run the workflow again

In [ ]:
atoms=bulk("Ag","fcc",4)
structure=AseAtomsAdaptor().get_structure(atoms)
maker_fullopt=OptimizeJob()
job1=maker_fullopt.make(structure=structure)
job2=get_ev_curve(structure=job1.output["optimized_structure"])
job3=get_results_ev_curve(list_structures=job2.output["optimized_structures"], list_energies=job2.output["energies"])
flow=Flow([job1,job2,job3],output=job3.output)

response=run_locally(flow)
print(response[flow.jobs[-1].uuid][1].output)

You have seen all basic functionalities to write workflows with jobflow. You can also transfer these workflows to [Fireworks](https://materialsproject.github.io/atomate2/user/fireworks.html) to run the jobs in parallel and/or on a cluster.

### Exercise
Please build a large maker for the whole workflow! You can find an example here: [https://github.com/materialsproject/atomate2/blob/main/src/atomate2/vasp/flows/lobster.py](https://github.com/materialsproject/atomate2/blob/main/src/atomate2/vasp/flows/lobster.py)

Then, apply it to different structures and elements: e.g., Al or other elements that work with EMT.

# Phonon workflow

We will now write a tiny workflow to compute phonons with Phonopy. We will only compute phonon densities of states and not band structures as finding relevant points in reciprocal space is possible but a lot of work to implement. If you would like to try this, you are invited to have fun with pymatgen. Again, we will use the EMT calculator to speed up calculations.

We will compute phonons with the finite differences approach. You can check out this reference in case you are interested: [https://onlinelibrary.wiley.com/doi/10.1002/anie.200906780](https://onlinelibrary.wiley.com/doi/10.1002/anie.200906780). We will optimize the crystal structure, then generate displaced structures. This will allow to compute force constants for the whole unit cell. Those force constants can then be fouriertransformed to a dynamical matrix that allows us to get frequencies and eigenvalues describing the vibrations in the crystal structure.

![workflow for phonons](workflow2.png)

For phonon runs, we need an additional static job that just returns forces based on the structure. We can reuse the optimization. But first, we will install phonopy. Phonopy will create all displacements that we need according to symmetry and can also do all the annoying computations that are just a pain to reimplement. There are also some modules in ase for phonons but they don't handle symmetry that well so far!

In [ ]:
%%capture
!pip install phonopy

Let's build a static job first. This time, we also need the forces ...

In [ ]:
@dataclass
class StaticJob(Maker):
    """
    Class to compute energy and forces for a structure

    Parameters
    ----------
    name
        Name of the job.
    """
    name: str = "EMT-StaticRun"

    @job
    def make(self, structure: Structure):
        adaptor = AseAtomsAdaptor()
        atoms = adaptor.get_atoms(structure)
        atoms.calc = EMT()
        results = {}
        results["input_structure"] = structure
        results["energy"] = atoms.get_potential_energy()
        results["force"] = atoms.get_forces()
        return results



Now, we need to build jobs that will create displacements for the finite displacement method based on an optimized Structure, run all static runs and then collect the results. It's important to remember that you have to wait for a job to be executed before you can do something with the result in another job! You might be able to collect results from a job that will be run (e.g., put the result in a list but you are actually only collecting references) but you cannot perform any kind of numerical operation on it. You can only do this in the next job! The reference as to be resolved first!

Let's start with the job that creates displacements:

In [ ]:
from phonopy import Phonopy
from phonopy.units import VaspToTHz
from pymatgen.io.phonopy import (
    get_phonopy_structure,
    get_pmg_structure,
)


@job
def create_displacements(structure: Structure, supercell_matrix):
    factor = VaspToTHz  # we need this conversion factor. We will rely on eV
    cell = get_phonopy_structure(structure)  # this transforms the structure into a format that phonopy can use
    phonon = Phonopy(
        cell,
        supercell_matrix,  # we need to compute forces for displaced supercells of our inital structure
        factor=factor,
    )
    phonon.generate_displacements()  # phonopy will generate the displacements

    supercells = phonon.supercells_with_displacements  # these are all displaced structures

    displacements = []
    for icell in supercells:
        displacements.append(get_pmg_structure(icell))  # we will collect all structures in a pymatgen structure format

    return displacements

Now, we will generate static jobs for all of the generated structures

In [ ]:
@job
def run_displacements(displacements):
    static_maker=StaticJob()
    runs=[]
    forces=[]
    for displacement in displacements:
        displacement_run=static_maker.make(displacement)
        forces.append(displacement_run.output["force"]) # this is possible but you cannot transform the array to a numpy array yet as the static_maker jobs haven't been run yet!
        runs.append(displacement_run)

    output={"forces": forces} #we will collect all forces from the runs.

    return Response(replace=Flow(runs, output))

Now, we will write a job that evaluates all results. We again need to recreate the correct phonopy settings.

In [ ]:
from pymatgen.phonon.plotter import PhononDosPlotter
from pymatgen.io.phonopy import (
    get_ph_dos
)


@job
def summarize_phonopy_results(structure: Structure, forces, supercell_matrix, kpoints=[10, 10, 10]):
    forces_set = []
    for force in forces:
        forces_set.append(np.array(force))

    factor = VaspToTHz
    cell = get_phonopy_structure(structure)
    phonon = Phonopy(
        cell,
        supercell_matrix,
        factor=factor,
    )
    phonon.generate_displacements()
    phonon.produce_force_constants(forces=forces_set)
    phonon.save("phonopy.yaml")
    phonon.run_mesh(kpoints)
    phonon.run_total_dos()
    phonon.write_total_dos(filename="phonon_dos.yaml")
    dos = get_ph_dos("phonon_dos.yaml")
    new_plotter_dos = PhononDosPlotter()
    new_plotter_dos.add_dos(label="total", dos=dos)
    new_plotter_dos.save_plot(filename="phonon_dos.eps")
    return {"PhononDOS": dos}







Now we connect the jobs with an optimzation

In [ ]:
from jobflow.managers.local import run_locally
from jobflow import Flow
from ase.build import bulk

from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core.structure import Structure
atoms=bulk("Ag","fcc",3.9)
supercell_matrix=[[2,0,0],[0,2,0],[0,0,2]]
structure=AseAtomsAdaptor().get_structure(atoms)
maker_fullopt=OptimizeJob()
job1=maker_fullopt.make(structure=structure)
job2=create_displacements(structure=job1.output["optimized_structure"], supercell_matrix=supercell_matrix)
job3=run_displacements(displacements=job2.output)
job4=summarize_phonopy_results(structure=job1.output["optimized_structure"], forces=job3.output["forces"], supercell_matrix=supercell_matrix)
flow=Flow([job1,job2,job3,job4],output=job4.output)

response=run_locally(flow)

new_plotter_dos = PhononDosPlotter()
new_plotter_dos.add_dos(label="total", dos=response[flow.jobs[-1].uuid][1].output["PhononDOS"])
new_plotter_dos.show()



## Exercise
Again, plug the flow into a maker. Then, build an even larger maker including 3 phonon runs at the optimized volume, one with a volume that is 1% smaller and one with a volume that is 1% larger. This is a typical setting to compute Grüneisen parameters.